In [1]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
# from langchain.llms import OpenAI
# from langchain.agents import load_tools, AgentType, Tool, initialize_agent
# os.environ["OPENAI_API_KEY"] = "ENter your API key here"

# warnings.filterwarnings("ignore")

In [ ]:
llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

In [71]:
import yfinance as yf

# Fetch stock data from Yahoo Finance

def get_stock_price(ticker,history=5):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    # print(df.columns)
    
    return df.to_string()

print(get_stock_price("Reliance"))

                  Close   Volume
Date                            
2024-04-09  2927.300049  3535707
2024-04-10  2959.149902  4569165
2024-04-12  2934.300049  7775506
2024-04-15  2929.649902  6451031
2024-04-16  2931.500000  4683092


In [5]:
def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    # time.sleep(4) #To avoid rate limit error
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)


    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("Hindustan Unilever"))

Recent News:

0. HUL Q4 results: Net profit falls 6% to Rs 2,406 cr; FMCG firm declares Rs 
24 dividend
1. HUL Q4 results: Net profit declines 6%, revenue marginally up; Here are 4 
key highlights | Mint
2. HUL Q4 dividend: FMCG giant Hindustan Unilever announces Rs 24 dividend



In [29]:

def get_financial_statements(ticker):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker+".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    print("running fin statements")
    
    # cash_flow = company.cash_flow.to_string()
    # print(balance_sheet)
    # print(cash_flow)
    return balance_sheet
print(get_financial_statements("TATAPOWER.NS"))

running fin statements
                                                                           2023-03-31       2022-03-31      2021-03-31
Ordinary Shares Number                                                   3196991847.0     3196991847.0    3195339547.0
Share Issued                                                             3196991847.0     3196991847.0    3195339547.0
Net Debt                                                               447847600000.0   445128500000.0  395012900000.0
Total Debt                                                             529230900000.0   511952100000.0  467082200000.0
Tangible Book Value                                                    255477800000.0   192170700000.0  191818400000.0
Invested Capital                                                       777619500000.0   700316500000.0  654931700000.0
Working Capital                                                       -130013600000.0  -141013900000.0 -119092800000.0
Net Tangible Assets      

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search=DuckDuckGoSearchRun()

search("Stock news India")

d:\Deepak\Deepak_CS\web\stockify\Stockify\.env\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [ ]:
# import openai

# import os
# #os.environ["OPENAI_API_KEY"] = "Enter your API key here"
# #openai.api_key="Enter your API key here"
# from langchain.llms import OpenAI

# llm=OpenAI(temperature=0,
#            model_name="gpt-3.5-turbo-16k-0613")

# #Openai function calling

# import json
# function=[
#         {
#         "name": "get_company_Stock_ticker",
#         "description": "This will get the indian NSE/BSE stock ticker of the company",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "ticker_symbol": {
#                     "type": "string",
#                     "description": "This is the stock symbol of the company.",
#                 },

#                 "company_name": {
#                     "type": "string",
#                     "description": "This is the name of the company given in query",
#                 }
#             },
#             "required": ["company_name","ticker_symbol"],
#         },
#     }
# ]



# def get_stock_ticker(query):
#     response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo",
#             temperature=0,
#             messages=[{
#                 "role":"user",
#                 "content":f"Given the user request, what is the comapany name and the company stock ticker ?: {query}?"
#             }],
#             functions=function,
#             function_call={"name": "get_company_Stock_ticker"},
#     )
#     message = response["choices"][0]["message"]
#     arguments = json.loads(message["function_call"]["arguments"])
#     company_name = arguments["company_name"]
#     company_ticker = arguments["ticker_symbol"]
#     return company_name,company_ticker

# get_stock_ticker("How is ths stock of Paytm doing?")


# def Anazlyze_stock(query):
#     #agent.run(query) Outputs Company name, Ticker
#     Company_name,ticker=get_stock_ticker(query)
#     print({"Query":query,"Company_name":Company_name,"Ticker":ticker})
#     stock_data=get_stock_price(ticker,history=10)
#     stock_financials=get_financial_statements(ticker)
#     stock_news=get_recent_stock_news(Company_name)

#     # available_information=f"Stock Price: {stock_data}\n\nStock Financials: {stock_financials}\n\nStock News: {stock_news}"
#     available_information=f"Stock Financials: {stock_financials}\n\nStock News: {stock_news}"

#     print("\n\nAnalyzing.....\n")
#     analysis=llm(f"Give detail stock analysis, Use the available data and provide investment recommendation. \
#              The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
#              User question: {query} \
#              You have the following information available about {Company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
#               {available_information} "
#              )
#     print(analysis)

#     return analysis

# out=Anazlyze_stock("Shall I invest in Adani power right now?")    

In [10]:
import os
import config
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

genai = ChatGoogleGenerativeAI(model="gemini-1.0-pro", google_api_key=config.api_key)
function = [
    {
        "name": "get_company_Stock_ticker",
        "description": "This will get the Indian NSE/BSE stock ticker of the company",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker_symbol": {
                    "type": "string",
                    "description": "This is the stock symbol of the company."
                },
                "company_name": {
                    "type": "string",
                    "description": "This is the name of the company given in the query."
                }
            },
            "required": ["company_name", "ticker_symbol"]
        }
    }
]

# Define the prompt template
prompt_template = """
Given the user request: {query}
What is the company name and the company stock ticker?

Respond in the following format:
Company Name - Ticker Symbol
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["query"])

# Create the LLMChain with the prompt and the Google Gemini API
llm_chain = LLMChain(prompt=prompt, llm=genai)

def get_stock_ticker(query):
    # Run the LLMChain with the user query
    response = llm_chain.run(query)

    # Parse the response to extract the company name and ticker symbol
    try:
        company_name, ticker_symbol = response.split(" - ")
    except ValueError:
        # Handle cases where the response is not in the expected format
        company_name = "Unknown"
        ticker_symbol = "Unknown"

    return company_name, ticker_symbol
get_stock_ticker("How is ths stock of Adani doing?")

('Adani', 'ADANIENT')

### PROMPT 

 that gives stock data and financial statements based on api


In [11]:

def Anazlyze_stock(query):
    Company_name, ticker = get_stock_ticker(query)
    print({"Query": query, "Company_name": Company_name, "Ticker": ticker})

    # Add your code to fetch stock data, financials, and news

    # Define the prompt template for stock analysis
    analyze_stock_prompt = """
    Given the user request: {query}
    Provide a detailed stock analysis for {Company_name} (Ticker: {ticker}). Use the available data and provide an investment recommendation.
    
    Available information:
    {available_information}
    """
    analyze_stock_prompt_template = PromptTemplate(template=analyze_stock_prompt, input_variables=["query", "Company_name", "ticker", "available_information"])
    analyze_stock_chain = LLMChain(prompt=analyze_stock_prompt_template, llm=genai)

    available_information = "Placeholder for stock data, financials, and news"
    analysis = analyze_stock_chain.run(query=query, Company_name=Company_name, ticker=ticker, available_information=available_information)

    print("\n\nAnalyzing.....\n")
    print(analysis)
    return analysis

out = Anazlyze_stock("Shall I invest in Adani power right now?")

{'Query': 'Shall I invest in Adani power right now?', 'Company_name': 'Adani Power', 'Ticker': 'ADANIPOWER'}


Analyzing.....

**Stock Analysis for Adani Power (ADANIPOWER)**

**Company Overview:**

Adani Power is a leading private power producer in India, with a total installed capacity of over 13,000 MW. The company operates thermal, solar, and wind power plants across the country.

**Stock Performance:**

* **Current Price:** ₹235.30
* **52-Week High:** ₹390.05
* **52-Week Low:** ₹148.50
* **Market Cap:** ₹92,757 crore

**Financials:**

* **Revenue (FY23):** ₹47,556 crore
* **Net Income (FY23):** ₹4,033 crore
* **EPS (FY23):** ₹7.63
* **P/E Ratio:** 30.8x

**Technical Analysis:**

* **Moving Averages:** The stock is currently trading below its 50-day and 200-day moving averages, indicating a bearish trend.
* **Relative Strength Index (RSI):** The RSI is at 32, which is in the oversold zone. This suggests that the stock may be due for a rebound.
* **Support and Resistance Levels:** T

In [ ]:

def Anazlyze_stock(query):
    Company_name, ticker = get_stock_ticker(query)
    print({"Query": query, "Company_name": Company_name, "Ticker": ticker})

    # Add your code to fetch stock data, financials, and news

    # Define the prompt template for stock analysis
    Company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"Company_name":Company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker,history=10)
    stock_financials=get_financial_statements(ticker)
    stock_news=get_recent_stock_news(Company_name)
    available_information="Stock Financials: {stock_financials}\n\nStock News: {stock_news}"
    analyze_stock_prompt='''Provide a detailed stock analysis for {Company_name} (Ticker: {ticker}). Use the available data and provide an investment recommendation.
     \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {Company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} '''
             
    analyze_stock_prompt_template = PromptTemplate(template=analyze_stock_prompt, input_variables=["query", "Company_name", "ticker", "available_information"])
    analyze_stock_chain = LLMChain(prompt=analyze_stock_prompt_template, llm=genai)

    available_information = "Placeholder for stock data, financials, and news"
    analysis = analyze_stock_chain.run(query=query, Company_name=Company_name, ticker=ticker, available_information=available_information)

    print("\n\nAnalyzing.....\n")
    print(analysis)
    return analysis

out = Anazlyze_stock("Shall I invest in Adani power right now?")

# ADJUSTED PARAMS AND PROMPT 
to read news 


In [62]:
def Anazlyze_stock(query):
    Company_name, ticker = get_stock_ticker(query)
    print({"Query": query, "Company_name": Company_name, "Ticker": ticker})

    # Add your code to fetch stock data, financials, and news

    # Define the prompt template for stock analysis
    Company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"Company_name":Company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker,history=10)
    stock_financials=get_financial_statements(ticker)
    stock_news=get_recent_stock_news(Company_name)
    available_information=f" Stock Data: {stock_data} \n\nStock Financials: {stock_financials}\n\nStock News: {stock_news}"
    analyze_stock_prompt=f'''Give detail stock analysis, Use the available data and provide investment recommendation.
     \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {Company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} '''
             
    analyze_stock_prompt_template = PromptTemplate(template=analyze_stock_prompt, input_variables=["query", "Company_name", "ticker", "available_information"])
    analyze_stock_chain = LLMChain(prompt=analyze_stock_prompt_template, llm=genai)

    available_information = "Placeholder for stock data, financials, and news"
    analysis = analyze_stock_chain.run(query=query, Company_name=Company_name, ticker=ticker, available_information=available_information)

    print("\n\nAnalyzing.....\n")
    
    analysis += ("\n\n"+stock_data+"\n\n")
    print(stock_news+"\n\n")
    print(analysis)
    return analysis



In [63]:
out = Anazlyze_stock("Shall i invest in Asian Paints?")


{'Query': 'Shall i invest in Asian Paints?', 'Company_name': 'Asian Paints', 'Ticker': 'APNT'}
{'Query': 'Shall i invest in Asian Paints?', 'Company_name': 'Asian Paints', 'Ticker': 'ASIANPAINT'}
running fin statements


Analyzing.....

Recent News:




**Investment Analysis:**

1. **Strong Revenue Growth:** Asian Paints has consistently reported strong revenue growth over the past several years, driven by increasing demand for its products in India and international markets.

2. **Expanding Margins:** The company has been able to expand its margins through cost control measures and efficient operations, leading to higher profitability.

3. **Strong Brand Recognition:** Asian Paints is one of the most well-known and trusted paint brands in India, with a strong market share in the decorative paint segment.

4. **Diversification:** The company has diversified its product portfolio beyond paints, including wallpapers, adhesives, and home decor products, providing additional growth opportu